In [2]:
import import_ipynb
from Task_2 import hamming_distance as HD
from lib import bn256
import random
import numpy as np
import copy

In [3]:
v1, v2 = (np.random.choice([-1, 1], 16) for _ in range(2))
v1, v2

(array([ 1, -1,  1,  1, -1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1]),
 array([-1, -1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1]))

In [4]:
G1 = bn256.curve_G
G2 = bn256.twist_G

In [5]:
v1points = [G2.scalar_mul(bn256.order + i) for i in v1]
v2points  = [G1.scalar_mul(bn256.order + i) for i in v2]

In [6]:
type(v1points[0]), type(v2points[0])

(lib.bn256.curve_twist, lib.bn256.curve_point)

In [7]:
h1, h2 = (random.randrange(2, bn256.order) for _ in range(2))
H1, H2 = G1.scalar_mul(h1), G2.scalar_mul(h2)
#H1, H2, h1, h2

In [8]:
def CIPE_setup(n):
    s, t = ([random.randrange(2, bn256.order) for i in range(n)] for _ in range(2))
    u, v = ([random.randrange(2, bn256.order) for i in range(n+2)] for _ in range(2))
    gen1_h = [G1.scalar_mul(s[i]).add(H1.scalar_mul(t[i])) for i in range(n)]
    gen2_h = [G2.scalar_mul(u[i]).add(H2.scalar_mul(v[i])) for i in range(n+2)]
    msk = (s, t, u, v, gen1_h, gen2_h)
    return msk

In [9]:
def Registration(G2=G2, msk, v1points):
    s, t, gen2_h = msk[0], msk[1], msk[5]
    n = len(s)
    r0 = random.randrange(2, bn256.order)
    reg_template = [None for _ in range(n+4)]
    reg_template[0] = G2.scalar_mul(r0)
    reg_template[1] = H2.scalar_mul(r0)
    for i in range(n+2):
        reg_template[i+2] = gen2_h[i].scalar_mul(r0)
    for i in range(n):
        vs = v1points[i].scalar_mul(s[i]).scalar_mul(bn256.order - 1)
        vt = v1points[i].scalar_mul(t[i]).scalar_mul(bn256.order - 1)
        reg_template[2] = reg_template[2].add(vs)
        reg_template[3] = reg_template[3].add(vt)
        reg_template[i+4] = reg_template[i+4].add(v1points[i])
    
    return reg_template

In [10]:
def Authentication(G1=G1, msk, v2points):
    u, v, gen1_h = msk[2], msk[3], msk[4]
    n = len(u)
    r0 = random.randrange(2, bn256.order)
    auth_template = [None for _ in range(n+2)]
    auth_template[0] = G1.scalar_mul(0)
    auth_template[1] = G1.scalar_mul(0)
    auth_template[2] = G1.scalar_mul(r0)
    auth_template[3] = H1.scalar_mul(r0)
    for i in range(n-2):
        auth_template[i+4] = gen1_h[i].scalar_mul(r0).add(v2points[i])
    for i in range(n):
        vu = auth_template[i+2].scalar_mul(u[i]).scalar_mul(bn256.order - 1)
        vv = auth_template[i+2].scalar_mul(v[i]).scalar_mul(bn256.order - 1)
        auth_template[0] = auth_template[0].add(vu)
        auth_template[1] = auth_template[1].add(vv)
    return auth_template

In [11]:
def Decryption(msk, v1points, v2points):
    reg_template = Registration(msk, v1points)
    auth_template = Authentication(msk, v2points)
    e = bn256.optimal_ate(reg_template[0], auth_template[0])
    n = len(auth_template)
    for i in range(1, n):
        e = e.mul(bn256.optimal_ate(reg_template[i], auth_template[i]))
    return reg_template, auth_template, e

In [12]:
msk = CIPE_setup(len(v1points))
e = Decryption(msk, v1points, v2points)[2]

In [13]:
pair = bn256.optimal_ate(G2, G1)
neg_pair = bn256.optimal_ate(G2, G1.scalar_mul(bn256.order - 1))
log_table = [pair.exp(i) for i in range(16, -1, -2)] + [neg_pair.exp(i) for i in range(2, 17, 2)]


In [14]:
HD(v1,v2), log_table.index(e)


(9.0, 9)

ЗІЙШЛОСЬ